In [1]:
import os
from pathlib import Path
import re
import pandas as pd

In [4]:
writer = pd.ExcelWriter("extracted_data_failed_blocksize.xlsx", engine='xlsxwriter')

# From Below!

In [5]:
output_folder = Path("../output_data/blocksize")

In [6]:
for file_path in os.listdir(output_folder):
    # Open the file
    with open(Path(output_folder)/Path(file_path), 'r') as file:
        lines = file.readlines()
        
        
    # Define regular expressions to match the required patterns
    bond_dim_pattern = re.compile(r'Bond dimension = (\d+)')
    time_elapsed_pattern = re.compile(r'Time elapsed = ([\d.]+)')
    energy_pattern = re.compile(r'E = ([\d.e+-]+)')
    delta_energy_pattern = re.compile(r'DE = ([\d.e+-]+)')
    
    # Initialize variables to store extracted values
    bond_dimension = None
    time_elapsed = None
    energy = None
    delta_energy = None
    bond_dimensions = []
    bd_line = []
    energies = []
    delta_energy = []
    after_sweep_lines = []
    # Find and extract the values
    count = 0
    in_relevant_section = True
    catalyst_name = ''
    for line in lines:
        
        # Get the Catalyst name
        if 'fcidump.'in line:
            index = line.find('fcidump.')  # Find the index of 'fcidump.'
            if index != -1:  # If 'fcidump.' is found in the line
                catalyst_name = line[index+ len('fcidump.'):index+ len('fcidump.') + 6] + line[-20: -1]  # Extract next 10 characters
        
        # Check if we are in the relevant section
        if line.startswith("Sweep =   19"):
            count += 1
            bd_line.append(line)
            in_relevant_section = True
        elif line.startswith("Sweep"):
            in_relevant_section = False
        elif line.startswith("Time sweep = "):
            in_relevant_section = False
        
        if in_relevant_section:
            if line.startswith("Time elapsed ="):
                after_sweep_lines.append(line)
                
    # Regular expression pattern to match Bond dimension value
    bond_dim_pattern1 = re.compile(r'Bond dimension = (\d+)')
    bond_dim_pattern2 = re.compile(r'Bond dimension =  (\d+)')
    bond_dim_pattern3 = re.compile(r'Bond dimension =   (\d+)')
    bond_dim_pattern4 = re.compile(r'Bond dimension =    (\d+)')
    bond_dim_pattern5 = re.compile(r'Bond dimension =     (\d+)')
    # List to store extracted Bond dimension values
    bond_dimensions = []
    
    if len(bd_line) > len(after_sweep_lines):
        bd_line.pop()
    
    # Extract Bond dimension values from each line
    for line in bd_line:
        bond_dim_match1 = bond_dim_pattern1.search(line)
        bond_dim_match2 = bond_dim_pattern2.search(line)
        bond_dim_match3 = bond_dim_pattern3.search(line)
        bond_dim_match4 = bond_dim_pattern4.search(line)
        bond_dim_match5 = bond_dim_pattern5.search(line)
        if bond_dim_match1:
            bond_dimension = int(bond_dim_match1.group(1))
            bond_dimensions.append(bond_dimension)
        if bond_dim_match2:
            bond_dimension = int(bond_dim_match2.group(1))
            bond_dimensions.append(bond_dimension)
        if bond_dim_match3:
            bond_dimension = int(bond_dim_match3.group(1))
            bond_dimensions.append(bond_dimension)
        if bond_dim_match4:
            bond_dimension = int(bond_dim_match4.group(1))
            bond_dimensions.append(bond_dimension)
        if bond_dim_match5:
            bond_dimension = int(bond_dim_match5.group(1))
            bond_dimensions.append(bond_dimension)
    
    # Print the extracted Bond dimension values
    print("Bond dimensions:", bond_dimensions)
    
    
    # Regular expression patterns to match E and DW values
    e_pattern = re.compile(r'E =\s+(-?\d+\.\d+)')
    dw_pattern = re.compile(r'DW =\s+(-?\d+\.\d+e[-+]\d+)')
    
    # Extract E and DE values
    E_values = []
    DW_values = []
    
    for line in after_sweep_lines:
        e_match = e_pattern.search(line)
        dw_match = dw_pattern.search(line)
        if e_match:
            E_values.append(float(e_match.group(1)))
        if dw_match:
            DW_values.append(float(dw_match.group(1)))
            
    df_1 = pd.DataFrame()
    column_names = ["DMRG Energy","Bond Dimension", "Discarded weights"]
    data = [E_values, bond_dimensions, DW_values]
    #Iteratively add data to new columns
    for column_name, column_data in zip(column_names, data):
        df_1[column_name] = column_data
    
    # Specify the path where you want to save the CSV file
    csv_path = f"../excel_data_storage/20240611performance_blocksize_failed/{catalyst_name}.csv"
    
    # Save DataFrame to a CSV file
    df_1.to_csv(csv_path, index=False)

Bond dimensions: [6, 7, 10, 15, 17, 19, 21, 24, 27, 30, 33, 37, 41, 46, 51, 57, 63, 70, 77, 85, 94, 104, 115, 127, 140, 154, 170, 188, 207, 228, 251, 277, 305, 336, 370, 408, 449, 494, 544, 599, 659, 725, 798, 878, 966, 1063, 1170, 1287, 1416, 1558, 1714, 1886, 2075, 2283, 2512, 2764, 3041]
Bond dimensions: [3, 4, 5, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 24, 27, 30, 33, 37, 41, 46, 51, 57, 63, 70, 77, 85, 94, 104, 115, 127, 140, 154, 170, 188, 207, 228, 251, 277, 305, 336, 370, 408, 449, 494, 544, 599, 659, 725]
Bond dimensions: [5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 24, 27, 30, 33, 37, 41, 46, 51, 57, 63, 70, 77, 85, 94, 104, 115, 127, 140, 154, 170, 188, 207, 228, 251, 277, 305, 336, 370, 408, 449, 494, 544, 599, 659]
Bond dimensions: [2, 4, 5, 6, 7, 8, 9, 10, 11, 13]
Bond dimensions: [3, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 24, 27, 30, 33, 37, 41, 46, 51, 57, 63, 70, 77, 85, 94, 104, 115, 127, 140, 154, 170, 188, 207, 228, 251, 277, 305, 336, 370, 408, 449, 494, 544, 599, 659]
B

In [7]:
writer.close()

In [6]:
# Open the file
with open(Path(output_folder)/Path(file_path), 'r') as file:
    lines = file.readlines()


In [7]:
print(len(lines))

128671


In [8]:
# Define regular expressions to match the required patterns
bond_dim_pattern = re.compile(r'Bond dimension = (\d+)')
time_elapsed_pattern = re.compile(r'Time elapsed = ([\d.]+)')
energy_pattern = re.compile(r'E = ([\d.e+-]+)')
delta_energy_pattern = re.compile(r'DE = ([\d.e+-]+)')

# Initialize variables to store extracted values
bond_dimension = None
time_elapsed = None
energy = None
delta_energy = None
bond_dimensions = []
bd_line = []
energies = []
delta_energy = []
after_sweep_lines = []
# Find and extract the values
count = 0
in_relevant_section = True
for line in lines:
    # Check if we are in the relevant section
    if line.startswith("Sweep =   19"):
        count += 1
        bd_line.append(line)
        in_relevant_section = True
    elif line.startswith("Sweep"):
        in_relevant_section = False
    elif line.startswith("Time sweep = "):
        in_relevant_section = False
    
    if in_relevant_section:
        if line.startswith("Time elapsed ="):
            after_sweep_lines.append(line)
        # bond_dim_match = bond_dim_pattern.search(line)
        # time_elapsed_match = time_elapsed_pattern.search(line)
        # energy_match = energy_pattern.search(line)
        # delta_energy_match = delta_energy_pattern.search(line)
        # 
        # if bond_dim_match:
        #     bond_dimensions.append(int(bond_dim_match.group(1)))
        # elif energy_match:
        #     energies.append(float(energy_match.group(1)))
        # elif delta_energy_match:
        #     delta_energy.append(float(delta_energy_match.group(1)))
print(f"Number of Sweep 19 : {count}")
# Print the extracted values
print("Bond dimension:", bond_dimensions)
print("Time elapsed:", time_elapsed)
print("Energy:", energies)
print("Delta Energy:", delta_energy)

Number of Sweep 19 : 50
Bond dimension: []
Time elapsed: None
Energy: []
Delta Energy: []


In [9]:
print(len(after_sweep_lines))

50


In [10]:
print(after_sweep_lines)

['Time elapsed =    119.541 | E =   -3676.7231625314 | DE = -6.93e-07 | DW = 9.41e-03\n', 'Time elapsed =    129.841 | E =   -3687.9573936774 | DE = -4.41e-02 | DW = 1.92e-02\n', 'Time elapsed =    157.093 | E =   -3707.1740620371 | DE = -2.34e-05 | DW = 4.56e-03\n', 'Time elapsed =    155.446 | E =   -3685.4576753783 | DE = 4.07e-06 | DW = 7.76e-03\n', 'Time elapsed =    179.688 | E =   -3737.6392663911 | DE = -2.08e-05 | DW = 4.54e-03\n', 'Time elapsed =    189.386 | E =   -3745.0315329419 | DE = -1.22e-05 | DW = 2.66e-03\n', 'Time elapsed =    166.953 | E =   -3729.2278266831 | DE = -3.43e-03 | DW = 1.05e-02\n', 'Time elapsed =    197.809 | E =   -3728.6877353849 | DE = -1.57e-02 | DW = 4.78e-03\n', 'Time elapsed =    205.402 | E =   -3748.7149769360 | DE = 1.27e-04 | DW = 5.64e-03\n', 'Time elapsed =    193.040 | E =   -3759.1145150012 | DE = -1.70e-04 | DW = 4.94e-03\n', 'Time elapsed =    200.387 | E =   -3721.2269970575 | DE = -3.06e-05 | DW = 6.47e-03\n', 'Time elapsed =    220

In [11]:
print(len(bd_line))

50


### Extract Bond dimension

In [139]:
# Regular expression pattern to match Bond dimension value
bond_dim_pattern1 = re.compile(r'Bond dimension = (\d+)')
bond_dim_pattern2 = re.compile(r'Bond dimension =  (\d+)')
bond_dim_pattern3 = re.compile(r'Bond dimension =   (\d+)')
bond_dim_pattern4 = re.compile(r'Bond dimension =    (\d+)')
bond_dim_pattern5 = re.compile(r'Bond dimension =     (\d+)')
# List to store extracted Bond dimension values
bond_dimensions = []

# Extract Bond dimension values from each line
for line in bd_line:
    bond_dim_match1 = bond_dim_pattern1.search(line)
    bond_dim_match2 = bond_dim_pattern2.search(line)
    bond_dim_match3 = bond_dim_pattern3.search(line)
    bond_dim_match4 = bond_dim_pattern4.search(line)
    bond_dim_match5 = bond_dim_pattern5.search(line)
    if bond_dim_match1:
        bond_dimension = int(bond_dim_match1.group(1))
        bond_dimensions.append(bond_dimension)
    if bond_dim_match2:
        bond_dimension = int(bond_dim_match2.group(1))
        bond_dimensions.append(bond_dimension)
    if bond_dim_match3:
        bond_dimension = int(bond_dim_match3.group(1))
        bond_dimensions.append(bond_dimension)
    if bond_dim_match4:
        bond_dimension = int(bond_dim_match4.group(1))
        bond_dimensions.append(bond_dimension)
    if bond_dim_match5:
        bond_dimension = int(bond_dim_match5.group(1))
        bond_dimensions.append(bond_dimension)

# Print the extracted Bond dimension values
print("Bond dimensions:", bond_dimensions)

Bond dimensions: [5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 24, 27, 30, 33, 37, 41, 46, 51, 57, 63, 70, 77, 85, 94, 104, 115, 127, 140, 154, 170, 188, 207, 228, 251, 277, 305, 336, 370, 408, 449, 494, 544, 599, 659, 725, 798, 878, 966, 1063, 1170, 1287]


In [140]:
# Regular expression patterns to match E and DW values
e_pattern = re.compile(r'E =\s+(-?\d+\.\d+)')
dw_pattern = re.compile(r'DW =\s+(-?\d+\.\d+e[-+]\d+)')

# Extract E and DE values
E_values = []
DW_values = []

for line in after_sweep_lines:
    e_match = e_pattern.search(line)
    dw_match = dw_pattern.search(line)
    if e_match:
        E_values.append(float(e_match.group(1)))
    if dw_match:
        DW_values.append(float(dw_match.group(1)))

# Print extracted values
print("E values:", E_values)
print(f"Lenth of E values {len(E_values)}")
print("DE values:", DW_values)
print(f"Lenth of DW values {len(DW_values)}")

E values: [-2188.8524933639, -2201.7684885021, -2219.8691395786, -2214.2083311983, -2226.2447061414, -2229.7578247683, -2231.7464241466, -2238.1702116775, -2246.9847749194, -2247.6934504468, -2253.0175747342, -2258.3700334081, -2266.6604697823, -2269.695620092, -2265.9482647469, -2266.4445924041, -2268.2492435841, -2271.9077069751, -2274.1737995937, -2283.0275306367, -2279.233663551, -2289.8625273071, -2291.0308243373, -2290.4724997219, -2292.3204784886, -2294.6274329576, -2295.3304065185, -2296.1987994283, -2297.8852612698, -2299.0079675446, -2300.3343344893, -2300.6931369755, -2301.5291615269, -2302.3015071029, -2303.1026410852, -2304.1414786278, -2304.7708515159, -2305.4222458414, -2305.9198257714, -2306.4383851267, -2306.8918788122, -2307.1111199066, -2307.2542187507, -2307.2342576021, -2307.2715641146, -2307.2681546613, -2307.2634185598, -2307.2694390245, -2307.2672138707, -2307.2718630989, -2307.264140851, -2307.2643672966, -2307.2698515021]
Lenth of E values 53
DE values: [0.075

In [141]:
df_1 = pd.DataFrame()
column_names = ["Bond dimension", "Energy", "DW"]
data = [bond_dimensions, E_values, DW_values]
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data):
    df_1[column_name] = column_data

df_1.to_excel(writer, sheet_name=catalyst_name, index=False)

# Stop!

In [142]:
writer.close()